In [2]:
import sys;sys.path.insert(0, '..')
from src.RMSE import BatchRMSE
import torch
import os
from src.FCN8MS import FCN8MS
import src.core as core
import src.guassian as guassian


cwd = os.getcwd()
FCN8_model_dir = os.path.join(cwd, '..', 'models', 'FCN8MS_FINAL_NO_AUG')

run_dir = os.path.join(FCN8_model_dir, 'run')
model_dir = os.path.join(FCN8_model_dir, 'model')


print(f"Using {core.TorchDevice} device")
torch.set_default_device(core.TorchDevice)

loaded_model = FCN8MS()
saved_model_path = model_path = os.path.join(model_dir, f'model_20240407_090921_8')
checkpoint = torch.load(saved_model_path, map_location=core.TorchDevice)
loaded_model.load_state_dict(checkpoint)
loaded_model.eval()

rmse = BatchRMSE()

predicted_heatmaps = None
single_sample = None
for i, sample in enumerate(core.TestDataSet):
    single_sample = sample
    image, heatmaps = sample
    image, heatmaps = image.to(core.TorchDevice), heatmaps.to(core.TorchDevice)

    ## We need to unsqueeze add the batch dimension to the image [1, 3, 96, 96]
    image = image.unsqueeze(0)
    # print("image.shape", image.shape)

    predicted_heatmaps = loaded_model(image)

    pred_heatmaps = predicted_heatmaps.cpu().detach()

    ## We need to remove the extra batch dimensions from the prediction [24, 96, 96]
    pred_heatmaps = pred_heatmaps.squeeze(0)

    # print("pred_heatmaps.shape", pred_heatmaps.shape)

    predicted_keypoints = guassian.heatmaps_to_keypoints_CoM(pred_heatmaps)

    ground_heatmaps = heatmaps.cpu().detach()

    gt_keypoints = guassian.heatmaps_to_keypoints_CoM(ground_heatmaps)

    rmse.add_pred_error(gt_keypoints, predicted_keypoints)


print(f"All Keypoints RMSE {rmse.get_all_keypoints_RMSE()}")



Using cpu device
All Keypoints RMSE 231.79416452824265


In [3]:
for i in range(24):
    try:
        print(f"All Keypoints RMSE {rmse.get_keypoint_RMSE(i)}")
    except:
        print(f"No RMSE for {i}th keypoint")

All Keypoints RMSE 10.489704760224335
All Keypoints RMSE 14.477959673647103
All Keypoints RMSE 8.86335385250195
All Keypoints RMSE 883.2329535105113
All Keypoints RMSE 121.14547018927252
All Keypoints RMSE 277.7420627031818
All Keypoints RMSE 16.109275215697288
All Keypoints RMSE 8.569732539612895
All Keypoints RMSE 12.502561044793916
All Keypoints RMSE 265.90812058592144
All Keypoints RMSE 316.7949238309719
All Keypoints RMSE 664.6396996769205
All Keypoints RMSE 13.571471654026412
All Keypoints RMSE 25.317723794282127
All Keypoints RMSE 39.65247157146489
All Keypoints RMSE 8.70307187611864
All Keypoints RMSE 8.629603572845658
All Keypoints RMSE 7.874319153171715
All Keypoints RMSE 19.53646916653954
All Keypoints RMSE 14.38327172895085
No RMSE for 20th keypoint
No RMSE for 21th keypoint
No RMSE for 22th keypoint
No RMSE for 23th keypoint
